**Question 01**

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch
import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

# Extract dataset
zip_path = "/content/drive/MyDrive/nature_12K.zip"
data_dir = "/content/inaturalist_12K"
if not os.path.exists(data_dir):
    !cp "{zip_path}" .
    !unzip -q nature_12K.zip
    !rm nature_12K.zip

# Function to compute dataset mean & std
def compute_mean_std(data_dir, input_size=224):
    transform = transforms.Compose([
        transforms.Resize((input_size, input_size)),
        transforms.ToTensor()
    ])
    dataset = datasets.ImageFolder(root=data_dir, transform=transform)
    loader = DataLoader(dataset, batch_size=64, shuffle=False, num_workers=0)  # Colab: num_workers=0

    sum_rgb = torch.zeros(3)
    sum_sq_rgb = torch.zeros(3)
    total_pixels = 0

    for images, _ in loader:
        batch_samples = images.size(0)
        pixels_per_image = images.size(2) * images.size(3)
        total_pixels += batch_samples * pixels_per_image

        sum_rgb += images.sum(dim=[0, 2, 3])
        sum_sq_rgb += (images ** 2).sum(dim=[0, 2, 3])

    mean = sum_rgb / total_pixels
    std = torch.sqrt((sum_sq_rgb / total_pixels) - (mean ** 2))

    return mean.tolist(), std.tolist()

# Compute and print
data_dir_train = os.path.join(data_dir, "train")
inat_mean, inat_std = compute_mean_std(data_dir_train)
print(f"Mean: {inat_mean}")
print(f"Std: {inat_std}")

Mounted at /content/drive
Mean: [0.47122955322265625, 0.46000856161117554, 0.3896463215351105]
Std: [0.24068380892276764, 0.23018933832645416, 0.2406243085861206]


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
import torchvision.transforms as transforms

class FlexibleCNN(pl.LightningModule):
    def __init__(self, input_channels=3, num_classes=10, conv_filters=[32, 64, 128, 256, 512],
                 kernel_size=3, activation=F.relu, dense_neurons=256, input_size=224):
        super(FlexibleCNN, self).__init__()

        self.activation = activation
        layers = []
        in_channels = input_channels

        # Adding 5 convolutional blocks
        for out_channels in conv_filters:
            layers.append(nn.Conv2d(in_channels, out_channels, kernel_size, padding=1))
            layers.append(nn.BatchNorm2d(out_channels))
            layers.append(nn.ReLU())
            layers.append(nn.MaxPool2d(2, 2))
            in_channels = out_channels

        self.conv_layers = nn.Sequential(*layers)

        # Compute final feature map size after 5 max-pool layers
        final_size = input_size // (2 ** len(conv_filters))

        # Fully connected layers
        self.fc1 = nn.Linear(conv_filters[-1] * final_size * final_size, dense_neurons)
        self.fc2 = nn.Linear(dense_neurons, num_classes)

        # Compute parameters and computations
        self.total_parameters = self.compute_parameters(conv_filters, kernel_size, dense_neurons, num_classes)
        self.total_computations = self.compute_computations(conv_filters, kernel_size, input_size)

        # Define preprocessing transform for iNaturalist dataset
        self.transform = transforms.Compose([
            transforms.Resize((input_size, input_size)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.47122955322265625, 0.46000856161117554, 0.3896463215351105],
                                 std=[0.24068380892276764, 0.23018933832645416, 0.2406243085861206])
        ])

    def forward(self, x):
        x = self.conv_layers(x)
        x = torch.flatten(x, start_dim=1)
        x = self.activation(self.fc1(x))
        x = self.fc2(x)
        return x

    def compute_parameters(self, conv_filters, kernel_size, dense_neurons, num_classes):
        total_params = 0
        in_channels = 3  # Input image channels

        # Compute parameters for convolution layers
        for out_channels in conv_filters:
            total_params += (in_channels * kernel_size * kernel_size + 1) * out_channels
            in_channels = out_channels

        # Compute parameters for fully connected layers
        total_params += (conv_filters[-1] * (224 // (2**5)) * (224 // (2**5))) * dense_neurons + dense_neurons
        total_params += dense_neurons * num_classes + num_classes

        return total_params

    def compute_computations(self, conv_filters, kernel_size, input_size):
        total_computations = 0
        in_channels = 3  # Input image channels
        feature_map_size = input_size

        # Compute computations for convolution layers
        for out_channels in conv_filters:
            feature_map_size //= 2  # Max-pooling reduces size by half
            total_computations += out_channels * feature_map_size * feature_map_size * (in_channels * kernel_size * kernel_size)
            in_channels = out_channels

        return total_computations

# Example usage:
model = FlexibleCNN(input_channels=3, num_classes=10, conv_filters=[32, 64, 128, 256, 512],
                    kernel_size=3, activation=F.relu, dense_neurons=256, input_size=224)
print(model)
print(f"Total Parameters: {model.total_parameters}")
print(f"Total Computations: {model.total_computations}")

FlexibleCNN(
  (conv_layers): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=

**Question 02**

In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ["PYDEVD_DISABLE_FILE_VALIDATION"] = "1"
import torch
import random
import wandb
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
import torchvision.transforms as T
from torch.utils.data import DataLoader, Subset
from torchvision.datasets import ImageFolder
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from google.colab import drive

# Set reproducibility seed
pl.seed_everything(42, workers=True)

# Mount Google Drive
drive.mount("/content/drive", force_remount=True)

# Dataset setup
zip_file = "/content/drive/MyDrive/nature_12K.zip"
extracted_dir = "/content/inaturalist_12K/train"

if not os.path.exists(extracted_dir):
    !cp "{zip_file}" .
    !unzip -q nature_12K.zip
    !rm nature_12K.zip

# Helper: Subsample dataset to keep it class-balanced
def create_class_balanced_subset(dataset, per_class=400):
    class_map = {i: [] for i in range(len(dataset.classes))}
    for idx, (_, label) in enumerate(dataset.samples):
        class_map[label].append(idx)
    chosen_indices = []
    for class_id, indices in class_map.items():
        chosen = random.sample(indices, min(per_class, len(indices)))
        chosen_indices.extend(chosen)
    random.shuffle(chosen_indices)
    return Subset(dataset, chosen_indices)

# Activation selector
ACT_FNS = {
    "ReLU": nn.ReLU(),
    "GELU": nn.GELU(),
    "SiLU": nn.SiLU(),
    "Mish": nn.Mish()
}

# CNN model with flexible design
class CustomCNN(pl.LightningModule):
    def __init__(self, cfg):
        super().__init__()
        self.save_hyperparameters()

        input_res = 224
        in_channels = 3
        output_classes = 10
        kernel = cfg.get("kernel_size")
        act_fn = ACT_FNS[cfg.get("activation")]
        use_bn = cfg.get("batch_norm")
        dropout_rate = cfg.get("dropout")
        filter_mode = cfg.get("filter_type")
        base = cfg.get("base_filter")
        hidden_units = cfg.get("dense_neurons")

        # Determine conv filters
        if filter_mode == "same":
            filters = [base] * 5
        elif filter_mode == "double":
            filters = [base * (2 ** i) for i in range(5)]
        else:
            filters = [max(4, base // (2 ** i)) for i in range(5)]

        conv_blocks = []
        for out_channels in filters:
            conv_blocks.append(nn.Conv2d(in_channels, out_channels, kernel, padding=1))
            if use_bn:
                conv_blocks.append(nn.BatchNorm2d(out_channels))
            conv_blocks.append(act_fn)
            conv_blocks.append(nn.MaxPool2d(2))
            in_channels = out_channels

        self.feature_extractor = nn.Sequential(*conv_blocks)

        # Get flattened shape
        with torch.no_grad():
            dummy = torch.zeros(1, 3, input_res, input_res)
            shape = self.feature_extractor(dummy).view(1, -1).shape[1]

        self.classifier = nn.Sequential(
            nn.Linear(shape, hidden_units),
            nn.Dropout(dropout_rate),
            nn.ReLU(),
            nn.Linear(hidden_units, output_classes)
        )

        self.criterion = nn.CrossEntropyLoss()

    def forward(self, x):
        x = self.feature_extractor(x)
        x = x.view(x.size(0), -1)
        return self.classifier(x)

    def training_step(self, batch, _):
        x, y = batch
        preds = self(x)
        loss = self.criterion(preds, y)
        acc = (preds.argmax(1) == y).float().mean()
        self.log("train_loss", loss, prog_bar=True)
        self.log("train_acc", acc, prog_bar=True)
        return loss

    def validation_step(self, batch, _):
        x, y = batch
        preds = self(x)
        loss = self.criterion(preds, y)
        acc = (preds.argmax(1) == y).float().mean()
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)

# Data module for PyTorch Lightning
class INatDataModule(pl.LightningDataModule):
    def __init__(self, cfg):
        super().__init__()
        self.batch = 32
        self.use_aug = cfg.get("augmentation")

    def setup(self, stage=None):
        mean, std = [0.4712, 0.4600, 0.3896], [0.2406, 0.2301, 0.2406]
        tfms = [T.Resize((224, 224))]
        if self.use_aug:
            tfms += [T.RandomHorizontalFlip(), T.RandomRotation(15)]
        tfms += [T.ToTensor(), T.Normalize(mean, std)]

        transform = T.Compose(tfms)
        full_dataset = ImageFolder(extracted_dir, transform=transform)
        balanced_subset = create_class_balanced_subset(full_dataset)

        val_len = int(0.2 * len(balanced_subset))
        train_len = len(balanced_subset) - val_len
        self.train_set, self.val_set = torch.utils.data.random_split(balanced_subset, [train_len, val_len])

    def train_dataloader(self):
        return DataLoader(self.train_set, batch_size=self.batch, shuffle=True, num_workers=2)

    def val_dataloader(self):
        return DataLoader(self.val_set, batch_size=self.batch, shuffle=False, num_workers=2)

# Train model with W&B sweep
def launch_training(config=None):
    with wandb.init(config=config):
        cfg = dict(wandb.config)
        wandb.run.name = "_".join([f"{k}-{v}" for k, v in cfg.items()])
        model = CustomCNN(cfg)
        data_module = INatDataModule(cfg)

        logger = WandbLogger(project="inat-sweep-v3", log_model=False)
        early_stop = EarlyStopping(monitor="val_acc", mode="max", patience=3)

        trainer = pl.Trainer(
            max_epochs=10,
            logger=logger,
            callbacks=[early_stop],
            accelerator="auto"
        )
        trainer.fit(model, datamodule=data_module)

# Sweep configuration
sweep_settings = {
    "method": "bayes",
    "metric": {"name": "val_acc", "goal": "maximize"},
    "parameters": {
        "base_filter": {"values": [64, 128, 256]},
        "kernel_size": {"values": [3, 5]},
        "activation": {"values": ["ReLU", "GELU", "SiLU", "Mish"]},
        "filter_type": {"values": ["same", "double", "half"]},
        "batch_norm": {"values": [True, False]},
        "augmentation": {"values": [True, False]},
        "dropout": {"values": [0, 0.1, 0.2]},
        "dense_neurons": {"values": [128, 256, 512]}
    }
}

# Run sweep
sweep_id = wandb.sweep(sweep_settings, project="inat-sweep-v3")
wandb.agent(sweep_id, function=launch_training, count=20)

INFO:lightning_fabric.utilities.seed:Seed set to 42


Mounted at /content/drive


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Create sweep with ID: 0bpldk8l
Sweep URL: https://wandb.ai/tejaswiniksssn-indian-institute-of-technology-madras/inat-sweep-v3/sweeps/0bpldk8l


wandb: Agent Starting Run: 3d3s1s5p with config:
wandb: 	activation: Mish
wandb: 	augmentation: False
wandb: 	base_filter: 128
wandb: 	batch_norm: True
wandb: 	dense_neurons: 256
wandb: 	dropout: 0
wandb: 	filter_type: half
wandb: 	kernel_size: 3
wandb: Currently logged in as: tejaswiniksssn (tejaswiniksssn-indian-institute-of-technology-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


...

wandb: ERROR Run qhvg0m7w errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "/usr/local/lib/python3.11/dist-packages/wandb/agents/pyagent.py", line 306, in _run_job
wandb: ERROR     self._function()
wandb: ERROR   File "<ipython-input-1-07491dc79b5e>", line 172, in launch_training
wandb: ERROR     trainer.fit(model, datamodule=data_module)
wandb: ERROR   File "/usr/local/lib/python3.11/dist-packages/pytorch_lightning/trainer/trainer.py", line 561, in fit
wandb: ERROR     call._call_and_handle_interrupt(
wandb: ERROR   File "/usr/local/lib/python3.11/dist-packages/pytorch_lightning/trainer/call.py", line 48, in _call_and_handle_interrupt
wandb: ERROR     return trainer_fn(*args, **kwargs)
wandb: ERROR            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
wandb: ERROR   File "/usr/local/lib/python3.11/dist-packages/pytorch_lightning/trainer/trainer.py", line 599, in _fit_impl
wandb: ERROR     self._run(model, ckpt_path=ckpt_path)
wandb: ERROR   File "/usr/local/lib/python3.11

INFO:pytorch_lightning.utilities.rank_zero:You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name              | Type             | Params | Mode 
---------------------------------------------------------------
0 | feature_extractor | Sequential       | 400 K  | train
1 | classifier        | Sequential       | 203 K  | train
2 | criterion         | CrossEntropyLoss | 0      | train
---------------------------------------------------------------
603 K     Trainable params
0         Non-trainable params
603 K     Total params
2.414  

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Traceback (most recent call last):
  File "<ipython-input-1-07491dc79b5e>", line 172, in launch_training
    trainer.fit(model, datamodule=data_module)
  File "/usr/local/lib/python3.11/dist-packages/pytorch_lightning/trainer/trainer.py", line 561, in fit
    call._call_and_handle_interrupt(
  File "/usr/local/lib/python3.11/dist-packages/pytorch_lightning/trainer/call.py", line 48, in _call_and_handle_interrupt
    return trainer_fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pytorch_lightning/trainer/trainer.py", line 599, in _fit_impl
    self._run(model, ckpt_path=ckpt_path)
  File "/usr/local/lib/python3.11/dist-packages/pytorch_lightning/trainer/trainer.py", line 1012, in _run
    results = self._run_stage()
              ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pytorch_lightning/trainer/trainer.py", line 1054, in _run_stage
    self._run_sanity_check()
  File "/usr/local/lib/python3.11/dist-pack

wandb: ERROR Run 3xh06x8h errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "/usr/local/lib/python3.11/dist-packages/wandb/agents/pyagent.py", line 306, in _run_job
wandb: ERROR     self._function()
wandb: ERROR   File "<ipython-input-1-07491dc79b5e>", line 172, in launch_training
wandb: ERROR     trainer.fit(model, datamodule=data_module)
wandb: ERROR   File "/usr/local/lib/python3.11/dist-packages/pytorch_lightning/trainer/trainer.py", line 561, in fit
wandb: ERROR     call._call_and_handle_interrupt(
wandb: ERROR   File "/usr/local/lib/python3.11/dist-packages/pytorch_lightning/trainer/call.py", line 48, in _call_and_handle_interrupt
wandb: ERROR     return trainer_fn(*args, **kwargs)
wandb: ERROR            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
wandb: ERROR   File "/usr/local/lib/python3.11/dist-packages/pytorch_lightning/trainer/trainer.py", line 599, in _fit_impl
wandb: ERROR     self._run(model, ckpt_path=ckpt_path)
wandb: ERROR   File "/usr/local/lib/python3.11

**Question 04**

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from torchvision.datasets import ImageFolder
from torchvision.transforms import Compose, Resize, ToTensor, Normalize, RandomHorizontalFlip, RandomRotation
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
import torch.nn.functional as F
from google.colab import drive

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Mount Google Drive
drive.mount("/content/drive", force_remount=True)

# Dataset setup
zip_file = "/content/drive/MyDrive/nature_12K.zip"
extracted_dir = "/content/inaturalist_12K"

if not os.path.exists(extracted_dir):
    !cp "{zip_file}" .
    !unzip -q nature_12K.zip
    !rm nature_12K.zip

# Activation selector
ACT_FNS = {
    "ReLU": nn.ReLU(),
    "GELU": nn.GELU(),
    "SiLU": nn.SiLU(),
    "Mish": nn.Mish()
}

class FlexibleCNN(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        input_res = 224
        in_channels = 3
        output_classes = 10
        kernel = cfg.get("kernel_size")
        act_fn = ACT_FNS[cfg.get("activation")]
        use_bn = cfg.get("batch_norm")
        dropout_rate = cfg.get("dropout")
        filter_mode = cfg.get("filter_type")
        base = cfg.get("base_filter")
        hidden_units = cfg.get("dense_neurons")

        if filter_mode == "same":
            filters = [base] * 5
        elif filter_mode == "double":
            filters = [base * (2 ** i) for i in range(5)]
        else:
            filters = [max(4, base // (2 ** i)) for i in range(5)]

        conv_blocks = []
        for out_channels in filters:
            conv_blocks.append(nn.Conv2d(in_channels, out_channels, kernel, padding=1))
            if use_bn:
                conv_blocks.append(nn.BatchNorm2d(out_channels))
            conv_blocks.append(act_fn)
            conv_blocks.append(nn.MaxPool2d(2))
            in_channels = out_channels

        self.feature_extractor = nn.Sequential(*conv_blocks)

        with torch.no_grad():
            dummy = torch.zeros(1, 3, input_res, input_res)
            shape = self.feature_extractor(dummy).view(1, -1).shape[1]

        self.classifier = nn.Sequential(
            nn.Linear(shape, hidden_units),
            nn.Dropout(dropout_rate),
            nn.ReLU(),
            nn.Linear(hidden_units, output_classes)
        )

    def forward(self, x):
        x = self.feature_extractor(x)
        x = x.view(x.size(0), -1)
        return self.classifier(x)

# --- CONFIG ---
best_config = {
    "base_filter": 64,
    "kernel_size": 3,
    "activation": "GELU",
    "filter_type": "same",
    "batch_norm": False,
    "augmentation": True,
    "dropout": 0,
    "dense_neurons": 512
}

train_transform = Compose([
    Resize((224, 224)),
    RandomHorizontalFlip(),
    RandomRotation(15),
    ToTensor(),
    Normalize(mean=[0.4712, 0.4600, 0.3896], std=[0.2406, 0.2301, 0.2406])
])

test_transform = Compose([
    Resize((224, 224)),
    ToTensor(),
    Normalize(mean=[0.4712, 0.4600, 0.3896], std=[0.2406, 0.2301, 0.2406])
])

train_path = "/content/inaturalist_12K/train"
test_path = "/content/inaturalist_12K/val"

train_data = ImageFolder(train_path, transform=train_transform)
test_data = ImageFolder(test_path, transform=test_transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

# --- TRAINING ---
model = FlexibleCNN(best_config).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

best_test_acc = 0.0

print("Starting Training...\n")
for epoch in range(1, 11):  # 10 epochs
    model.train()
    total_loss = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)

    # --- Evaluation on test set ---
    model.eval()
    test_preds, test_labels = [], []
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            outputs = model(images)
            preds = outputs.argmax(dim=1).cpu().numpy()
            test_preds.extend(preds)
            test_labels.extend(labels.numpy())

    test_acc = accuracy_score(test_labels, test_preds)
    print(f"Epoch {epoch} | Avg Loss: {avg_loss:.4f} | Test Accuracy: {test_acc:.2%}")

    if test_acc > best_test_acc:
        best_test_acc = test_acc
        torch.save({"state_dict": model.state_dict()}, "best_model.ckpt")
        print(" Best model saved!\n")

# --- EVALUATION & VISUALIZATION ---
def test_model_performance(model, data_path, class_labels, max_vis_samples=30):
    model.eval()

    transform = test_transform
    test_data = ImageFolder(data_path, transform=transform)
    test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

    all_outputs, all_targets = [], []
    visual_samples = {"images": [], "preds": [], "labels": []}
    collected = 0

    with torch.no_grad():
        for batch_images, batch_labels in test_loader:
            batch_images = batch_images.to(device)
            logits = model(batch_images)
            predictions = logits.argmax(dim=1)

            all_outputs.extend(predictions.cpu().tolist())
            all_targets.extend(batch_labels.cpu().tolist())

            if collected < max_vis_samples:
                needed = min(max_vis_samples - collected, batch_images.size(0))
                visual_samples["images"].append(batch_images[:needed].cpu())
                visual_samples["preds"].append(predictions[:needed].cpu())
                visual_samples["labels"].append(batch_labels[:needed].cpu())
                collected += needed

    accuracy = accuracy_score(all_targets, all_outputs)
    print(f"\n Final Test Accuracy: {accuracy:.2%}")

    vis_images = torch.cat(visual_samples["images"])
    vis_preds = torch.cat(visual_samples["preds"])
    vis_labels = torch.cat(visual_samples["labels"])

    return accuracy, vis_images, vis_preds, vis_labels

def show_predictions_grid(images, predictions, labels, class_labels):
    mean = torch.tensor([0.4712, 0.4600, 0.3896]).reshape(3, 1, 1)
    std = torch.tensor([0.2406, 0.2301, 0.2406]).reshape(3, 1, 1)
    images = torch.clamp(images * std + mean, 0, 1)

    fig, axes = plt.subplots(10, 3, figsize=(13, 30))
    axes = axes.flatten()

    for idx in range(len(images)):
        ax = axes[idx]
        img = images[idx].permute(1, 2, 0).numpy()
        pred_label = class_labels[predictions[idx]]
        true_label = class_labels[labels[idx]]
        ax.imshow(img)
        ax.axis('off')
        ax.set_title(
            f"Pred: {pred_label}\nTrue: {true_label}",
            color="green" if predictions[idx] == labels[idx] else "red"
        )

    plt.tight_layout()
    plt.savefig("predicted_samples_grid.png", bbox_inches='tight')
    plt.show()

# --- Load Best Model & Run Evaluation ---
best_model = FlexibleCNN(best_config).to(device)
checkpoint_data = torch.load("best_model.ckpt", map_location=device)
best_model.load_state_dict(checkpoint_data["state_dict"])

label_names = ImageFolder(test_path).classes
final_acc, preview_imgs, preview_preds, preview_labels = test_model_performance(best_model, test_path, label_names)
show_predictions_grid(preview_imgs, preview_preds, preview_labels, label_names)

FileNotFoundError: [Errno 2] No such file or directory: '/content/inaturalist_12K/train'

In [ ]:
import nbformat
from google.colab import drive
import os

drive.mount('/content/drive')
# Path to current notebook
notebook_path = '/content/drive/MyDrive/PartA.ipynb'  # Change this!

# Load the notebook
with open(notebook_path, 'r', encoding='utf-8') as f:
    nb = nbformat.read(f, as_version=4)

# Remove metadata.widgets
if 'widgets' in nb['metadata']:
    del nb['metadata']['widgets']
    print("Removed 'metadata.widgets'")
else:
    print("No 'metadata.widgets' found")

# Save back to the same file (or change filename)
with open(notebook_path, 'w', encoding='utf-8') as f:
    nbformat.write(nb, f)
    print("Saved cleaned notebook")

Mounted at /content/drive
Removed 'metadata.widgets'
Saved cleaned notebook
